# NYC Service Requests

### You've been asked to perform data analysis of service request (311) calls from New York City. You've also been asked to utilize data wrangling techniques to understand the pattern in the data and visualize the major types of complaints.

In [1]:
# import dependencies
import numpy as np 
import pandas as pd
import matplotlib.pyplot as plt
%matplotlib inline
import seaborn as sns
from matplotlib.pyplot import figure
import scipy.stats as stats

In [2]:
df=pd.read_csv("311_Service_Requests_from_2010_to_Present.csv", low_memory=False)

In [3]:
# first 5 rows to visualize
df.head()

,Unique Key,Created Date,Closed Date,Agency,Agency Name,Complaint Type,Descriptor,Location Type,Incident Zip,Incident Address,...,Bridge Highway Name,Bridge Highway Direction,Road Ramp,Bridge Highway Segment,Garage Lot Name,Ferry Direction,Ferry Terminal Name,Latitude,Longitude,Location
0,32310363,12/31/2015 11:59:45 PM,01/01/2016 12:55:15 AM,NYPD,New York City Police Department,Noise - Street/Sidewalk,Loud Music/Party,Street/Sidewalk,10034.0,71 VERMILYEA AVENUE,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,40.865682,-73.923501,"(40.86568153633767, -73.92350095571744)"
1,32309934,12/31/2015 11:59:44 PM,01/01/2016 01:26:57 AM,NYPD,New York City Police Department,Blocked Driveway,No Access,Street/Sidewalk,11105.0,27-07 23 AVENUE,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,40.775945,-73.915094,"(40.775945312321085, -73.91509393898605)"
2,32309159,12/31/2015 11:59:29 PM,01/01/2016 04:51:03 AM,NYPD,New York City Police Department,Blocked Driveway,No Access,Street/Sidewalk,10458.0,2897 VALENTINE AVENUE,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,40.870325,-73.888525,"(40.870324522111424, -73.88852464418646)"
3,32305098,12/31/2015 11:57:46 PM,01/01/2016 07:43:13 AM,NYPD,New York City Police Department,Illegal Parking,Commercial Overnight Parking,Street/Sidewalk,10461.0,2940 BAISLEY AVENUE,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,40.835994,-73.828379,"(40.83599404683083, -73.82837939584206)"
4,32306529,12/31/2015 11:56:58 PM,01/01/2016 03:24:42 AM,NYPD,New York City Police Department,Illegal Parking,Blocked Sidewalk,Street/Sidewalk,11373.0,87-14 57 ROAD,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,40.733060,-73.874170,"(40.733059618956815, -73.87416975810375)"


In [ ]:
# reload in the data and change date columns to datetime
df=pd.read_csv("311_Service_Requests_from_2010_to_Present.csv", parse_dates = ['Created Date','Closed Date','Due Date'], infer_datetime_format = True, low_memory=False)

In [ ]:
# checking new date columns
df.head()

In [ ]:
#info and column names
df.info()

In [ ]:
# size/shape of data
df.shape

## Missing value treatment

In [ ]:
# checking missing/null values
df.isnull().sum().sort_values(ascending=False)

In [ ]:
# % of missing values
df.isnull().sum().sort_values(ascending=False)*100/364558

In [ ]:
df2=pd.DataFrame(df.columns.to_list()).set_index(0)
df2

In [ ]:
df2=df2[df.isnull().sum()*100/364558 < 80 ].reset_index()
df2

In [ ]:
# school has a lot of unspecified values
df3=df[df2[0].to_list()]
df3

In [ ]:
#confirming that df3 has dropped all the categories with more than 80% missing values
df3.isnull().sum().sort_values(ascending=False)*100/364558

In [ ]:
df3.shape

In [ ]:
#columns went from 53 to 39 after removing nulls
df3.info()

In [ ]:
# all of school code are unspecified
df3['School Code'].value_counts()

In [ ]:
# all but one of school name are unspecified
df3['School Name'].value_counts()

In [ ]:
df3[df3['School Name'] != 'Unspecified']

In [ ]:
# remove all school related columns if they have 'unspecified'
removed_school_list = [col for col in df3.columns.to_list() if col.find('School')<0]
removed_school_list

In [ ]:
# made new dataset without any 'School' related columns
pd.set_option('display.max_columns', None)
df4=df3[removed_school_list]
df4.head(10)

In [ ]:
# remaining 29 columns
df4.info()

In [ ]:
# Borough and community board have similar values so drop 'community board'
df5 = df4.drop(columns = ['Community Board'])
df5

In [ ]:
# creating new column as time elapsed between request creation and closing (to show how fast the response time is)
df5['Closing Time'] = df5['Closed Date'] - df5['Created Date'] #calculating time taken

In [ ]:
df5.head()

In [ ]:
df5['Time Bin'] = pd.cut(df5['Closing Time'], 100)#

In [ ]:
df5.info()

In [ ]:
df5['Closing Time Sec'] = df5['Closing Time'].apply(lambda x:x.seconds)

In [ ]:
df5['Closing Time Sec']

In [ ]:
df5.info()

## Finding major types of complaints

In [ ]:
# bar graph of count vs. complaint types
df5['Complaint Type'].value_counts().plot(kind='bar',figsize=(10,6),title='Most Frequent Complaints')
plt.show()

In [ ]:
# showing top 10 most amount of complaints with blocked driveway and illegal parking as the most by a lot
df5['Complaint Type'].value_counts().head(10)

In [ ]:
# Showing how many complaints per Borough but some are unspecified
df5['Borough'].value_counts()

In [ ]:
# removing unspecified complaints
df5.replace('Unspecified',np.NaN,inplace=True)
df5=df5.dropna(how='all')

In [ ]:
# confirming unspecified are removed
df5['Borough'].value_counts()

In [ ]:
# pie plot showing Borough wise percentage of complaints and that Brooklyn followed by Queens have the most
df5['Borough'].value_counts().plot(kind='pie',autopct='%.2f%%',figsize=(10,6),title='Complaints Per Borough')
plt.show()

## Visualize the major types of complaints in each city

In [ ]:
# Group the data by city and complaint type, and count the number of complaints
grouped_data = df5.groupby(['City', 'Complaint Type']).size().reset_index(name='count')

# Pivot the data to create a matrix with cities as rows, complaint types as columns, and complaint counts as values
pivoted_data = grouped_data.pivot(index='City', columns='Complaint Type', values='count')

# Create a bar plot with stacked bars for each city, showing the distribution of complaint types
pivoted_data.plot(kind='bar', stacked=True, figsize=(10, 6))

# Add labels and titles to the plot
plt.xlabel('City')
plt.ylabel('Number of Complaints')
plt.title('Distribution of Complaint Types by City')
plt.legend(title='Complaint Type', bbox_to_anchor=(1.05, 1), loc='upper left')
plt.show()

In [ ]:
# Calculate the total number of complaints for each city
city_totals = pivoted_data.sum(axis=1)

# sort the cities by this value and pick the top 5 (the first 5 elements of the index, from the beginning up to (but not including) the element at index 5)
sorted_cities = city_totals.sort_values(ascending=False).index[:5]

# Filter the pivoted data to show only the top 5 cities and their most common complaint types
pd1 = pivoted_data.loc[sorted_cities]
pd2 = pd1.loc[pd1.sum(axis=1).sort_values(ascending=False).index[:5]].T

# Create a bar plot with stacked bars for each city, showing the distribution of complaint types
pd2.plot(kind='bar', stacked=True, figsize=(10, 6))

# Add labels and titles to the plot
plt.xlabel('City')
plt.ylabel('Number of Complaints')
plt.title('Distribution of Complaint Types by City (Top 5)')
plt.legend(title='Complaint Type', bbox_to_anchor=(1.05, 1), loc='upper left')
plt.show()

In [ ]:
# Get the value counts for each city
city_counts = df['City'].value_counts()

# Create a horizontal bar chart
plt.figure(figsize=(8, 10))
plt.barh(city_counts.index, city_counts.values)

# Add labels and titles to the plot
plt.xlabel('Number of Complaints')
plt.ylabel('City')
plt.title('Number of Complaints by City')

# Adjust the padding between subplots to prevent overlapping labels
plt.tight_layout()
plt.show()

In [ ]:
# get the frequency of complaints by city
complaint_freq = df5['City'].value_counts()

# create a bar plot of the complaint frequencies
fig, ax = plt.subplots(figsize=(10, 8))

# the x-axis values are top 5 complaint types (extracted from the index using .index) and y-axis values are frequency of each complaint type (extracted from the complaint_freq series)
ax.bar(complaint_freq.index[:5], complaint_freq[:5])

# Add labels and titles to the plot
ax.set_xlabel('City')
ax.set_ylabel('Number of Complaints')
ax.set_title('Top 5 Cities by Number of Complaints')
plt.show()

In [ ]:
# select only the rows of df5 where the borough is 'BROOKLYN'
brooklyn_data = df5[df5['Borough'] == 'BROOKLYN']

# scatter plot of complaint locations in Brooklyn
fig, ax = plt.subplots(figsize=(20,10))
ax.scatter(brooklyn_data['Longitude'], brooklyn_data['Latitude'], s=1, alpha=0.1)

# Add labels and titles to the plot
ax.set_xlabel('Longitude')
ax.set_ylabel('Latitude')
ax.set_title('Complaint Locations in Brooklyn (Scatter Plot)')
plt.show()

# Create a hexbin plot of complaint concentrations in Brooklyn
fig, ax = plt.subplots(figsize=(20,10))
hb = ax.hexbin(brooklyn_data['Longitude'], brooklyn_data['Latitude'], gridsize=50, cmap='inferno')

# Add a colorbar and labels and titles to the plot
cb = plt.colorbar(hb, ax=ax)
cb.set_label('Number of Complaints')
ax.set_xlabel('Longitude')
ax.set_ylabel('Latitude')
ax.set_title('Complaint Concentrations in Brooklyn (Hexbin Plot)')
plt.show()

## Identify significant variables by performing a statistical analysis using p-values and chi-square values

In [ ]:
# create contingency table of the variables we want to test
cont_table = pd.crosstab(df5['Complaint Type'], df5['Borough'])

In [ ]:
# chi-square test of independence to calculate the p-value and chi-square value
chi2, p, dof, expected = stats.chi2_contingency(cont_table)

In [ ]:
# print out the p-value and chi-square value
print("p-value:", p)
print("chi-square:", chi2)

#### -Null Hypothesis: H0 - there is NO significant relation between type of complaint and location
#### -Alternate Hypothesis: Ha - there IS significant relation between type of complaint and location

## Results

### p-value 0.00 < 0.05
### -REJECT null hypothesis
### -complaint type and location ARE related